In [2]:
import torch

In [3]:
torch.cuda.is_available()

True

In [4]:
import pandas as pd
df = pd.read_csv(r"E:\Projects\encoder - Decoder - chatbot\qajokes1.1.2.csv")
df.head(5)

,Question,Answer,dirt,humor,kwid,meta
0,What's the best anti diarrheal prescription?,Mycheexarphlexin,0.0,0.2,BEST-ANTI-DIARRHEAL-PRESCRIPTION-MYCHEEXARPHLEXIN,NaN
1,What do you call a person who is outside a doo...,Matt,0.0,0.2,PERSON-OUTSIDE-DOOR-ARMS-LEGS-MATT,NaN
2,Which Star Trek character is a member of the m...,Jean-Luc Pickacard,0.0,0.2,STAR-TREK-CHARACTER-MEMBER-MAGIC-CIRCLE-JEANLU...,NaN
3,What's the difference between a bullet and a h...,A bullet doesn't miss Harambe,0.0,0.2,DIFFERENCE-BULLET-HUMAN-BULLET-DOESNT-MISS-HAR...,NaN
4,Why was the Ethiopian baby crying?,He was having a mid-life crisis,0.0,0.2,ETHIOPIAN-BABY-CRYING-MIDLIFE-CRISIS,NaN


In [5]:
df = df.drop(columns=['kwid' , 'meta', 'dirt' , 'humor'])
df = df.dropna()
df = df.sample(frac=1).reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75107 entries, 0 to 75106
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Question  75107 non-null  object
 1   Answer    75107 non-null  object
dtypes: object(2)
memory usage: 1.1+ MB


In [6]:
df.sample(5)

,Question,Answer
36485,How would you describe frankenstein's birth?,Shocking!
46953,Why couldn't the Soviet Union get anything done?,They were always either Russian or Stalin.
57794,What do you call it when a homosexual asks a q...,A Query.
67802,What's the least popular documentary?,Jiro nightmares of assrape
29710,Why was the lizard's wife unsatisfied?,Her hubby had a reptile dysfunction.


In [7]:
#combine convokit dataset also
df2 = pd.read_csv(r"E:\Projects\encoder - Decoder - chatbot\Convokit_dataset.csv")
df2 = df2.drop(columns=['Unnamed: 0'])
df2 = df2.dropna()
df2 = df2.sample(frac=1).reset_index(drop=True)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221282 entries, 0 to 221281
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   questions  221282 non-null  object
 1   answers    221282 non-null  object
dtypes: object(2)
memory usage: 3.4+ MB


In [8]:
#concatinating both datsets
# Rename columns in df2 to match df
df2 = df2.rename(columns={'questions': 'Question', 'answers': 'Answer'})

# Concatenate both datasets
dfn = pd.concat([df, df2], ignore_index=True)

dfn = dfn.sample(frac=1).reset_index(drop=True)

dfn.head()

,Question,Answer
0,I'm coming.,Hurry up and get dressed.
1,Why did sumo wrestlers start shaving their legs?,To stop getting confused as feminists
2,What's brown and sticky?,Poo!
3,Oh! I'm sorry! I'm sorry! What have you got fo...,"Put it down, I said! It's nothing for you."
4,Please wait till my husband gets home.,Why not?


In [9]:
dfn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296389 entries, 0 to 296388
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   Question  296389 non-null  object
 1   Answer    296389 non-null  object
dtypes: object(2)
memory usage: 4.5+ MB


Tokenization

In [10]:
PAD_token = 0 #used for padding short sentences
SOS_token = 1 #start of sentence token
EOS_token = 2 #End-of-sentence token

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD" , SOS_token:"SOS", EOS_token:"EOS"}
        self.num_words = 3 #count sos , eos , pad

    def addSentence(self , sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self , word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word]=1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] +=1

In [11]:
import unicodedata
import re

# Define contractions dictionary
good_prefixes = {
    "i'm": "i am",
    "he's": "he is",
    "she's": "she is",
    "it's": "it is",
    "that's": "that is",
    "what's": "what is",
    "where's": "where is",
    "who's": "who is",
    "don't": "do not",
    "doesn't": "does not",
    "didn't": "did not",
    "can't": "cannot",
    "won't": "will not",
    "isn't": "is not",
    "aren't": "are not",
    "wasn't": "was not",
    "weren't": "were not",
    "i'll": "i will",
    "you'll": "you will",
    "we'll": "we will",
    "they'll": "they will",
}

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = " ".join([good_prefixes[word] if word in good_prefixes else word for word in s.split()])
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()    
    return s

# Read query / response pairs and return a voc object
def readVocs(datafile , corpus_name):
    print("Reading lines......")
    
    # Read the file and split into lines
    lines = open(datafile , encoding='utf-8').read().strip().split('\n')

    #split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('<CoSe')] for l in lines]
    voc = Voc(corpus_name)
    return voc , pairs


# Returns true iff both sentences in a pair 'p' are under the MAX_LENGTH threshold
MAX_LENGTH = 15
def filterPair(p):
    #input sequences need to preserve the last word for EOS token 
    return len(p[0].split(' '))<MAX_LENGTH and len(p[1].split(' '))<MAX_LENGTH

# Filter pairs using filterPair condition
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

#using the functions defined above , return a populated voc object and pairs list
def loadPrepareData(corpus_name , datafile , save_dir):
    print("Start preparing training data .... ")
    voc , pairs = readVocs(datafile , corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words....")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words: ", voc.num_words)
    return voc , pairs

Batchwise input and data preparation

In [12]:
import itertools

PAD_token = 0  # used for padding short sentences
SOS_token = 1  # start of sentence token
EOS_token = 2  # End-of-sentence token

def indexesFromSentence(voc , sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]

def zeroPadding(l , fillvalue = PAD_token):
    return list(itertools.zip_longest(*l , fillvalue = fillvalue))

def binaryMatrix(l , value=PAD_token):
    m = []
    for i , seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# returns padded input sequence tensor and lengths 
def inputVar(l , voc):
    indexes_batch = [indexesFromSentence(voc , sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar , lengths

# returns padded target sequence tensor, padding mask , and max target length
def outputVar(l , voc):
    indexes_batch = [indexesFromSentence(voc , sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.ByteTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar , mask , max_target_len

#returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
        print("Input sentence-",pair[0])
        print("Output sentece-",pair[1])
        print("-----------------------")
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len


In [13]:
import random

# Create a vocabulary instance
voc = Voc('test')

# Create pairs from your existing dataframe and normalize them
print("Creating pairs from dataframe...")
pairs = []
for _, row in dfn.iterrows():
    question = normalizeString(row['Question'])
    answer = normalizeString(row['Answer'])
    pairs.append([question, answer])

# Build vocabulary from pairs
print("Building vocabulary...")
for pair in pairs:
    voc.addSentence(pair[0])
    voc.addSentence(pair[1])

# Example for validation
print("\nTesting with a small batch...")
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("\nBatch information:")
print("Input shape:", input_variable.shape)
print("Lengths:", lengths)
print("Output shape:", target_variable.shape)

Creating pairs from dataframe...
Building vocabulary...

Testing with a small batch...
Input sentence- not often but it is possible . most people get scared when they see the shadow of their limits . they do not know how long the shadow really is . they do not know how far away the real limits are standing . . . they stop out of fear .
Output sentece- . . .when you work with clients on machines do they sometimes just jump up a level or two ? do something they did not know they were capable of ?
-----------------------
Input sentence- enough to get to jersey . i will walk the rest of the way . i ve been sitting a long time . nice meeting you . it is been a trip .
Output sentece- the bus ll pick you up over there . uh . . . you got enough bread for a ticket ?
-----------------------
Input sentence- you might say i am part of the landscape here . they call me ben kenobi .
Output sentece- who are you anyway ?
-----------------------
Input sentence- what couldn t the lifeguard save the hipp

In [14]:
def filterPair(voc, pair):
	# Return True if all words in both sentences are in vocab
	return all(word in voc.word2index for word in pair[0].split()) and \
		   all(word in voc.word2index for word in pair[1].split())

# Get valid pairs
valid_pairs = [pair for pair in pairs if filterPair(voc, pair)]

small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(valid_pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("\nInput variable:")
print("input shape:", input_variable.shape)
print("lengths:", lengths)
print("\nTarget variable:")
print("Output shape:", target_variable.shape)
print("mask shape:", mask.shape)
print("max_target_len:", max_target_len)

Input sentence- i still cannot believe they re sending you to the belt you of all people never meant to be born on a mission to discover the origin of life .
Output sentece- a day or so .
-----------------------
Input sentence- look get down now you fucking drunk ! you bum !
Output sentece- oh there you are honey . we were waiting for you . what happened ?
-----------------------
Input sentence- no it is not mr . dowd .
Output sentece- or this is this a significant difference ?
-----------------------
Input sentence- he was at the campground yesterday .
Output sentece- but you re sure ?
-----------------------
Input sentence- you tell me .
Output sentece- not everything is imputed to memory . the most confidential stuff is kept top secret hard copy . why would the zero file be kept secret ?
-----------------------

Input variable:
input shape: torch.Size([31, 5])
lengths: tensor([31, 12,  9,  8,  5])

Target variable:
Output shape: torch.Size([28, 5])
mask shape: torch.Size([28, 5])
ma

Architecture

In [16]:
import torch.nn as nn
class EncoderRNN(nn.Module):
    def __init__(self , hidden_size, embedding, n_layers=1,dropout=0):
        super(EncoderRNN,self).__init__()
        self.hidden_size = hidden_size
        self.embedding = embedding

#Initialize GRU ; THE input_size and hidden_size params are both set to 'hidden_size'ArithmeticError
#because our input size is a word embedding with number of features == hidden_size

        self.gru = nn.GRU(hidden_size,hidden_size,n_layers, dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths , hidden=None):
        
        #convert word indexes to embeddings
        embedded = self.embedding(input_seq)
        print(f"Shape after embedding: {embedded.shape}")
        
        #pack padded batch of sequences for RNN modu;e
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        print(f"Shape after packing: {packed.data.shape}")
        
        #forward pass through GRU
        outputs , hidden = self.gru(packed , hidden)
        print(f"Shape after GRU: {outputs.data.shape}")
        
        #unpack padding
        outputs , _ = torch.nn.utils.rnn.pad_packed_sequence(outputs)
        print(f"Shape after unpacking: {outputs.shape}")
        
        #sum birectional GRU  outputs
        outputs = outputs[:,:,:self.hidden_size]+outputs[:,:,self.hidden_size]+outputs[:,:,self.hidden_size:]
        print(f"Shape after summing bidirectional outputs: {outputs.shape}")
        
        #return output and final hidden state
        return outputs, hidden